In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

In [22]:
import requests
import json
from pandas.io.json import json_normalize
from itertools import islice
import geopandas as gpd
from shapely import wkt

In [4]:
# Envelope rond de provinciegrenzen doorzoeken. Alle objectids die daar binnen vallen downloaden en op een rijtje zetten.
envelope = '93003.99999999802+464251.5000000022+154328.6700000043+580999.9999999997' # Polygoon van NH trekt hij niet.
# In deze query wordt wegdeel_v gedownload. Dat is laag nummer 112. Aanpassen voor andere dataset.
laag = 'https://geoservices.noord-holland.nl/ags/rest/services/oi_op/oi_areaal/MapServer/52'

In [5]:
laaginfo = requests.get(laag+ '?f=pjson')
info = json_normalize(json.loads(laaginfo.content))
info_df = json_normalize(info.fields[0])
fieldsdf = info_df.name
fields = fieldsdf.tolist()
fields

['objectid',
 'materiaaltype',
 'ad_id',
 'verwerkingsstatus',
 'objectbegintijd',
 'objecteindtijd',
 'beheerder',
 'typespec',
 'naam',
 'lengte',
 'opmerking',
 'shape',
 'st_length(shape)']

In [6]:
query = laag +'/query?where=&text=&objectIds=&time=&geometry='+ envelope + '&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=&returnGeometry=false&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=true&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&f=pjson'
r =  requests.get(query, verify=False)
df = json_normalize(json.loads(r.content))
df.objectIds[0][0]
objectIds_list = df.objectIds[0]

/home/dataiku/dss/code-envs/python/Gerda/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geoservices.noord-holland.nl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [7]:
# Velden kiezen die je wilt downloaden en in het juiste format zetten of alles houden.
#fields = ['SHAPE','WEGBEHSRT','WEGNUMMER','WEGDEELLTR','HECTO_LTTR','BAANSUBSRT','RPE_CODE', 'ADMRICHTNG', 'RIJRICHTNG','STT_NAAM','WPSNAAMNEN','OBJECTID_1','OBJECTID']
fieldsstring = ''
for f in range(0,len(fields)):
    field = fields[f]
    if fieldsstring == '':
        fieldsstring = field
    else:
        fieldsstring = fieldsstring + '%2C+' + field

In [8]:
# objectIds opsplitsen in chunks
def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())
n = 100 # grootte van chunks
objectIds_chunks = list(chunk(objectIds_list, n))
aantalchunks = len(objectIds_chunks)

In [14]:
# Areaaldata downloaden per chunk
df_PNH = pd.DataFrame()
for i in range(0,aantalchunks):
    chunk = objectIds_chunks[i]
    objectids = ''
    for j in range(0,len(chunk)):
        objectid = str(chunk[j])
        if objectids == '':
            objectids = objectid
        else:
            objectids = objectids + '%2C+' + objectid
    query2=laag+'/query?where=&text=&objectIds='+ objectids +'&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields='+ fieldsstring + '&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&f=pjson'
    r =  requests.get(query2, verify=False)
    df2 = json_normalize(json.loads(r.content))
    data = json_normalize(df2.features[0])
    data['ID'] = 52
    df_PNH = df_PNH.append(data)

/home/dataiku/dss/code-envs/python/Gerda/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geoservices.noord-holland.nl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/dataiku/dss/code-envs/python/Gerda/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geoservices.noord-holland.nl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [15]:
len(df_PNH)

112

In [16]:
df_PNH.head()

,attributes.ad_id,attributes.beheerder,attributes.lengte,attributes.materiaaltype,attributes.naam,attributes.objectbegintijd,attributes.objecteindtijd,attributes.objectid,attributes.opmerking,attributes.st_length(shape),attributes.typespec,attributes.verwerkingsstatus,geometry.paths,ID
0,AD.DCB154123D544BD28E06F927D352BBF5,None,10.492085,None,None,1614038400000,None,1,N531,10.492085,betonnen paddentunnel,Actueel,"[[[104425.95589999855, 508570.3016999997], [10...",52
1,AD.84B1EF8675894337ADB3B478C85A7F07,None,11.464783,None,None,1614038400000,None,2,"Geen geleiding maar wel nodig, locatie bij ben...",11.464783,kunststof buis,Actueel,"[[[117564.03770000115, 534900.3539000005], [11...",52
2,AD.643E6803E4544262B6CBDB7925393E8E,None,34.253816,None,None,1614038400000,None,3,Mag ook een looprichel onder de brug komen ; N244,34.253816,stalen buis,Actueel,"[[[114562.1074000001, 507857.87359999865], [11...",52
3,AD.306917A60B2C4B3394754926C93D8306,None,18.404365,None,None,1614038400000,None,4,N239,18.404365,betonnen buis met loopplank,Actueel,"[[[125149.71249999851, 529681.6933000013], [12...",52
4,AD.5B03FA4DBB674926AB464A5C07C467CF,None,23.110641,None,None,1561420800000,None,5,N236,23.110641,betonnen paddentunnel,Actueel,"[[[137881.00349999964, 474646.2340000011], [13...",52


In [1]:
#geometry 
df_PNH['geometry.paths'] = df_PNH['geometry.paths'].apply(wkt.loads)

In [0]:
# Example: load a DSS dataset as a Pandas dataframe
mydataset = dataiku.Dataset("mydataset")
mydataset_df = mydataset.get_dataframe()

In [0]:
# Recipe outputs
all_road_objects = dataiku.Dataset("All_road_objects")
all_road_objects.write_with_schema(df_PNH)